In [36]:
import librosa
import IPython.display as ipd
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
from glob import glob
from scipy import signal
import random
import cv2
import os

## Dataset

In [67]:
def load_audio(filename=None, sr=16000, second=3, samples=None):
    samples, sample_rate = librosa.load(filename, sr=sr)
    
    if second is not None and len(samples) < sr * second:
        samples = np.pad(samples, (0, sr * second - len(samples)), 'constant')
    if second is not None and len(samples) > sr * second:
        samples = samples[0:sr * second]
    return samples


def predict_voice(sample):
    beat = librosa.beat.beat_track(sample)
    v_max = sample.max()
    w = beat[0] + 250*v_max
    
    if w > 270:
        label = 'angry'
    elif w < 140:
        label = 'thinking'
    else:
        label = 'normal'
    return label


def play(sample):
    return ipd.Audio(sample, 16000, autoplay=True)


In [65]:
files = os.listdir('./voice/')
audios = [(load_audio('./voice/' + f, 16000), f[:-5]) for f in files]


In [69]:
preds = [predict_voice(audios[i][0]) for i in range(len(audios))] 

D:\anaconda\lib\site-packages\librosa\util\utils.py:1377: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data_agg[idx_agg] = aggregate(data[idx_in], axis=axis)
D:\anaconda\lib\site-packages\scipy\fftpack\basic.py:160: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  z[index] = x
D:\anaconda\lib\site-packages\librosa\core\audio.py:447: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`,

['angry',
 'angry',
 'angry',
 'angry',
 'angry',
 'normal',
 'normal',
 'normal',
 'normal',
 'normal',
 'thinking',
 'thinking',
 'thinking',
 'normal',
 'thinking']

## 语谱图

In [ ]:
librosa.feature.melspectrogram(y=y, sr=sr)
librosa.display.specshow(librosa.power_to_db(librosa.stft(y),ref=np.max), y_axis='mel', x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel spectrogram')
plt.tight_layout()
plt.show()

## 录音/保存

In [ ]:
import pyaudio
import wave
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 3  #设置录音的时间长度
WAVE_OUTPUT_FILENAME = "./voice/thinking5.wav"
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)
print("* recording")
frames = []
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)
print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()
wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

## 画FFT图

In [ ]:
from tkinter import *
import wave
from scipy.fftpack import fft,ifft
import matplotlib.pyplot as plt
import numpy as np
 
def data_fft(data, time, time_start, time_end):
        #短时fft。截取一段时间内的数据先
        #time_start是开始时间，time_end是结束时间
        t = []
        y = []
        count = 0
        #for i in time:
        for i in range(time.size):
                if((time[i] >= time_start) & (time[i] <= time_end)):
                        count = count + 1
                        t = np.append(t, time[i])
                        y = np.append(y, data[0][i])    #只提取左声道
        #print (count)
                        
        yy=fft(y)                  #快速傅里叶变换
        yreal = yy.real               # 获取实数部分
        yimag = yy.imag               # 获取虚数部分
 
                
        yf=abs(fft(y))                # 取绝对值
        yf1=abs(fft(y))/len(t)           #归一化处理
        yf2 = yf1[range(int(len(t)/2))]  #由于对称性，只取一半区间
 
        xf = np.arange(len(y))        # 频率
        xf1 = xf
        xf2 = xf[range(int(len(t)/2))]  #取一半区间
 
        #plt.figure()
        """
        
        plt.subplot(221)
        plt.plot(t, y)   
        plt.title('Original wave')
 
        plt.subplot(222)
        plt.plot(xf,yf,'r')
        plt.title('FFT of Mixed wave(two sides frequency range)',fontsize=7,color='#7A378B')  #注意这里的颜色可以查询颜色代码表
 
        plt.subplot(223)
        plt.plot(xf1,yf1,'g')
        plt.title('FFT of Mixed wave(normalization)',fontsize=9,color='r')
 
        plt.subplot(224)
        plt.plot(xf2,yf2,'b')
        plt.title('FFT of Mixed wave)',fontsize=10,color='#F08080')
        
        """
        plt.plot(t, y)   
        plt.title('Original wave')
 
 
        plt.show()
        
        
def main():
	wave_data, time = read_wave_data('./voice/angry1.wav')
	
	data_fft(wave_data, time, 0, 2)
	

main()